In [ ]:
# Import statements
!pip install opencv-python

import numpy as np
import argparse
import cv2
import os

In [ ]:
# Paths to load the model
DIR = r"C:\Users\Gaurav"
PROTOTXT = os.path.join(DIR, r"colorize/colorization_deploy_v2.prototxt")
POINTS = os.path.join(DIR, r"colorize/pts_in_hull.npy")
MODEL = os.path.join(DIR, r"colorize/colorization_release_v2.caffemodel")



In [ ]:
# Load the model
print("Loading model...")
net = cv2.dnn.readNetFromCaffe(PROTOTXT, MODEL)
pts = np.load(POINTS)

# Load centers for ab channel quantization used for rebalancing.
class8 = net.getLayerId("class8_ab")
conv8 = net.getLayerId("conv8_313_rh")
pts = pts.transpose().reshape(2, 313, 1, 1)
net.getLayer(class8).blobs = [pts.astype("float32")]
net.getLayer(conv8).blobs = [np.full([1, 313], 2.606, dtype="float32")]

# Load the input image
image_path = r"C:\Users\Gaurav\Downloads\Black_and_white.jpg"
image = cv2.imread(image_path)
scaled = image.astype("float32") / 255.0
lab = cv2.cvtColor(scaled, cv2.COLOR_BGR2LAB)

resized = cv2.resize(lab, (224, 224))
L = cv2.split(resized)[0]
L -= 50

print("Colorizing the image...")
# Set the input for the network by creating a blob from the image L channel
net.setInput(cv2.dnn.blobFromImage(L))
# Perform forward pass through the network and obtain the predicted ab channels
ab = net.forward()[0, :, :, :].transpose((1, 2, 0))
# Resize the predicted ab channels to match the size of the original image
ab = cv2.resize(ab, (image.shape[1], image.shape[0]))

L = cv2.split(lab)[0]
# Concatenate the L channel with the predicted ab channels
colorized = np.concatenate((L[:, :, np.newaxis], ab), axis=2)
#Convert to BGR color space
colorized = cv2.cvtColor(colorized, cv2.COLOR_LAB2BGR)
#Clip within (0,1) range
colorized = np.clip(colorized, 0, 1)

colorized = (255 * colorized).astype("uint8")

#Display the original and colorized image
cv2.imshow("Original", image)
cv2.imshow("Colorized", colorized)

#Wait for a key press and then close all windows
cv2.waitKey(0)
cv2.destroyAllWindows()

Loading model...
Colorizing the image...


In [ ]:
# Load the model
print("Loading model...")
net = cv2.dnn.readNetFromCaffe(PROTOTXT, MODEL)
pts = np.load(POINTS)

# Load centers for ab channel quantization used for rebalancing.
class8 = net.getLayerId("class8_ab")
conv8 = net.getLayerId("conv8_313_rh")
pts = pts.transpose().reshape(2, 313, 1, 1)
net.getLayer(class8).blobs = [pts.astype("float32")]
net.getLayer(conv8).blobs = [np.full([1, 313], 2.606, dtype="float32")]

# Load the input video
video_path = r"C:\Users\Gaurav\Downloads\Video_frame.mp4"
output_path = r"C:\Users\Gaurav\Downloads\colorized_video.mp4"

cap = cv2.VideoCapture(video_path)
fps = cap.get(cv2.CAP_PROP_FPS)
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fourcc = cv2.VideoWriter_fourcc(*"mp4v")
out = cv2.VideoWriter(output_path, fourcc, fps, (frame_width, frame_height))


# Loop through the frames of the video until there are no more frames
# Perform colorization on each frame and display the original and colorized frames
# The loop can be exited by pressing 'q' key
while True:

    ret, frame = cap.read()

    if not ret:
        break

    scaled = frame.astype("float32") / 255.0
    lab = cv2.cvtColor(scaled, cv2.COLOR_BGR2LAB)
    resized = cv2.resize(lab, (224, 224))
    L = cv2.split(resized)[0]
    L -= 50

    net.setInput(cv2.dnn.blobFromImage(L))
    ab = net.forward()[0, :, :, :].transpose((1, 2, 0))
    ab = cv2.resize(ab, (frame.shape[1], frame.shape[0]))

    L = cv2.split(lab)[0]
    colorized = np.concatenate((L[:, :, np.newaxis], ab), axis=2)
    colorized = cv2.cvtColor(colorized, cv2.COLOR_LAB2BGR)
    colorized = np.clip(colorized, 0, 1)
    colorized = (255 * colorized).astype("uint8")

    out.write(colorized)

    cv2.imshow("Original", frame)
    cv2.imshow("Colorized", colorized)
    if cv2.waitKey(1) & 0xFF == ord("q"):
        break

cap.release()
out.release()
cv2.destroyAllWindows()

print("Colorization completed. Colorized video saved to", output_path)


Loading model...
Colorization completed. Colorized video saved to C:\Users\Gaurav\Downloads\colorized_video.mp4
